# Vector boson fusion in IDM

The final notebook is [Significance_countours.ipynb](./Significance_countours.ipynb) which is based on [Sensitivity_fit.ipynb](Sensitivity_fit.ipynb) which is based on [Sensitivity_plots.ipynb](Sensitivity_plots.ipynb) by JD

### Initialization

In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
import sys
import subprocess
import numpy as np
import pandas as pd
##pip3 install pyslha
#import pyslha # not longer required
import tempfile
import os
import re
def grep(pattern,multilinestring):
    '''Grep replacement in python
    as in: $ echo $multilinestring | grep pattern
    dev: re.M is for multiline strings
    '''
    import re 
    grp=re.finditer('(.*)%s(.*)' %pattern, multilinestring,re.M)
    return '\n'.join([g.group(0) for g in grp])

def subprocess_line_by_line(*args,TRUST_ERRORS=True,**kwargs):
    '''
    Subprocess output line by line. Stop of error found when TRUST_ERRORS=True, and simply
    report wait method otherwise.
    
    The arguments are the same as for the Popen constructor.
    
    WARNING: Works only in Python 3
    
    See: https://stackoverflow.com/a/28319191/2268280 
    and: https://stackoverflow.com/a/17698359/2268280
    
    Example:
    
    subprocess_line_by_line('for i in $(seq 1 3);do echo $i; sleep 1;done',shell=True)
    '''
    
    if not TRUST_ERRORS:
        kwargs['stderr']=subprocess.PIPE
        
    kwargs['stdout']=subprocess.PIPE
    kwargs['bufsize']=1
    kwargs['universal_newlines']=True
    s=subprocess.Popen(*args,**kwargs)
    with s as p:
        for line in p.stdout:
            print(line, end='') # process line here
    
    if TRUST_ERRORS:
        if p.returncode != 0:
            raise subprocess.CalledProcessError(p.returncode, p.args)
    else:
        return s.wait()
    
##Main madGRAPH script:
def preamble(MHc=750,MH0=110):
    return '''import model InertDoublet_UFO
define p = g u c d s b u~ c~ d~ s~ b~
define j = p  
define l+ = e+ mu+ 
define l- = e- mu- 
define vl = ve vm vt 
define vl~ = ve~ vm~ vt~

generate p p > h2 h2 j j @0

output ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL

'''

def lamL_loop(LHA_input_file,MHc=750,MH0=110,lamL=0.01):
    return '''launch ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL
0    
../'''+LHA_input_file+'''
../Cards/run_card.dat
set wa0 auto
set whch auto 
set lamL '''+str(lamL)+'''
set mmh0 '''+str(MH0)+'''
set mma0 '''+str(MHc)+'''
set mmhch '''+str(MHc)+'''
0

''' 


def closing(MHc=750,MH0=110):
    return '''launch ../studies/IDM/BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lambdaL -i
print_results --path=./result_BP_'''+str(int(MHc))+'_'+str(int(MH0))+'''_vs_lamdaL_mh0_110.txt --format=short


done
'''

### Input variables

In [3]:
thisroot='/home/restrepo/prog/ROOT/root/bin/thisroot.sh'
#thisroot='/opt/root5/bin/thisroot.sh'
MHc=750
MH0=240
if MH0%1!=0:
    sys.exit('ERROR: MH0 must be integer')
MADGRAPH='madgraph'
run_dir='Task_Asana'
work_dir='studies/IDM/'+run_dir
work_script='BP_'+str(int(MHc))+'_A_'+str(int(MH0))+'.txt'
LHA_input_file='MadGraph_cards/benchmarks/param_card_template.dat'
output_dir='studies/IDM/BP_'+str(int(MHc))+'_'+str(int(MH0))+'_vs_lambdaL' #madGRAPH output

### Running flags

In [4]:
CLONE_GIT_REPO=False #WARNING: Try to overwrite current contents!
INSTALL=False # If True check full installation
TEST=False #take a long time
VERBOSE=True #Print shell commands output line by line
if not INSTALL:
    CLONE_GIT_REPO=False

## Install root 5

### Prerequisites 
```bash
apt-get install cmake git dpkg-dev make g++ gcc binutils libx11-dev libxpm-dev \
libxft-dev libxext-dev gfortran libssl-dev libpcre3-dev \
xlibmesa-glu-dev libglew1.5-dev libftgl-dev \
libmysqlclient-dev libfftw3-dev libcfitsio-dev \
graphviz-dev libavahi-compat-libdnssd-dev \
libldap2-dev python-dev python3-dev libxml2-dev libkrb5-dev \
libgsl0-dev libqt4-dev r-base r-base-dev
```

Install ROOT 5 in some `PATH`
```bash
git clone http://root.cern.ch/git/root.git
cd root
git checkout v5-34-00-patches
./configure
make

```
Add to your `.bashrc`
```bash
source PATH/root/bin/thisroot.sh
```

In [5]:
f=open('kk.sh','w')
f.write('source '+thisroot+'\n')
f.write('which root\n')
f.close()

if not subprocess.Popen('bash kk.sh'.split(),
            stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()[0]:
    sys.exit('INSTALL ROOT: see instrucctions in notebook')

# Development of module to calculate one specific point

If `CLONE_GIT_REPO=False`, it is assumed that you did the clone as:
```bash
git clone  --recursive git@github.com:restrepo/VBF_IDM.git
```

In [6]:
if CLONE_GIT_REPO:  
    REPO='VBF_IDM'
    REPO_url='git@github.com:restrepo'
    main_dir='.' #WARNING: Try to overwirte contents
    if os.path.exists(main_dir+'index.ipynb'):
        sys.exit('ERROR: Repo files already exists. Check main_dir')
    if not os.path.isdir(main_dir):
        s=subprocess.Popen(['mkdir','-p',main_dir],stdout=subprocess.PIPE,stderr=subprocess.PIPE).wait()

    td=tempfile.mkdtemp()
    s=subprocess_line_by_line(('git clone  --recursive '+REPO_url+'/'+REPO+'.git').split(),cwd=td,
                 stdout=subprocess.PIPE,stderr=subprocess.PIPE,TRUST_ERRORS=False)

    s=subprocess.Popen('mv '+td+'/'+REPO+'/*  '+main_dir,shell=True,
                   stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

    s=subprocess.Popen('mv '+td+'/'+REPO+'/.* '+main_dir,shell=True,
                   stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

    os.rmdir(td+'/'+REPO)
    os.rmdir(td)

In [7]:
if INSTALL:
    s=subprocess.Popen('git checkout -b v2.3.3'.split(),cwd='madgraph',
                      stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()
    if 'Switched' not in s[1].decode('utf-8'):
        sys.exit('Submodule problems')

subprocess does not use .bashrc

In [8]:
f=open('madgraph/kk.sh','w')
f.write('source '+thisroot+'\n')
f.write('./bin/mg5_aMC install.dat\n')
f.close()

In [9]:
if INSTALL:
    if VERBOSE:
        subprocess_line_by_line('bash kk.sh'.split(),cwd='madgraph', TRUST_ERRORS=False )
    else:
        s=subprocess.call('bash kk.sh'.split(),cwd='madgraph', stdout=open('kk','w'),stderr=open('kkk','w') )

In [10]:
if TEST:
    if VERBOSE:
        subprocess_line_by_line('./test.sh'.split(), cwd='test',
            stdout=subprocess.PIPE,stderr=subprocess.PIPE)
    else:
    s=subprocess.Popen('./test.sh'.split(), cwd='test',
            stdout=subprocess.PIPE,stderr=subprocess.PIPE).communicate()

s[0].decode('utf-8')

### Create MadGraph script with processes

In [11]:
LambdasL=[0.01,0.02,0.05,0.07,0.1,0.15,0.2,0.25,0.3,0.35,0.4,0.45,0.5,0.55,0.6,1.0,3.0,5.0,7.0,10.0]

```bash
rsiii@gfif:~/IDM_VBF/Samples/mh0_vs_lal/scan$ ls delphes_events_[0-9][0-9]_*.root | more
delphes_events_70_001_.root
delphes_events_70_002_.root
delphes_events_70_003_.root
...
...
delphes_events_90_018_.root
delphes_events_90_019_.root
delphes_events_90_020_.root
```

```bash
rsiii@gfif:~/IDM_VBF/Samples/mh0_vs_lal/scan$ ls delphes_events_[0-9][0-9][0-9]_*.root | more
delphes_events_110_001_.root
delphes_events_110_002_.root
delphes_events_110_003_.root
...
...
delphes_events_240_018_.root
delphes_events_240_019_.root
delphes_events_240_020_.root
```

cat readme.txt 
date 23/01/2017
Delphes output (.root) for the set the points contained in data_mh0_vs_x.dat

The data is organized as follows (in ascending order in lambda_L[lal] ):

#lal #mh0[GeV] #xs[fb]   #Delphes name
0.3 63.75 200        --> delphes_events_2.root
0.4 64.80 200        --> delphes_events_3.root 
0.5 66.32 200        --> delphes_events_4.root
0.6 68.05 200        --> delphes_events_5.root 
0.7 70.04 200        --> delphes_events_6.root
.   .     .          --> 
.   .     .          --> 
.   .     .          -->

The cross-section (xs) for the processes p p > h0 h0 j j
for all the set of points is around 200 fb. [Except for
the first point, with lal=0.3,  where the cross-section turns out to be 179 fb.]

In [12]:
s=subprocess.Popen(['mkdir','-p',work_dir],
                 stdout=subprocess.PIPE, stderr=subprocess.PIPE).communicate()

In [14]:
f=open(work_dir+'/'+work_script,'w')
f.write( preamble(MHc,MH0) )
for lamL in LambdasL:
    f.write( lamL_loop(LHA_input_file,MHc,MH0,lamL=lamL) )
f.write( closing(MHc,MH0) )
f.close()

```bash
# pip3 install subprocess.run && #pip2 install subprocess.run 
```

In [15]:
f=open(MADGRAPH+'/kk.sh','w')
f.write('source '+thisroot+'\n')
f.write('./bin/mg5_aMC ../'+work_dir+'/'+work_script+'\n')
f.close()

In [25]:
if not VERBOSE:
    s=subprocess.Popen( 'bash kk.sh'.split(), cwd=MADGRAPH,
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    s.wait()

In [ ]:
if VERBOSE:
    subprocess_line_by_line( 'bash kk.sh'.split(), cwd=MADGRAPH,TRUST_ERRORS=False)

************************************************************
*                                                          *
*                     W E L C O M E to                     *
*              M A D G R A P H 5 _ a M C @ N L O           *
*                                                          *
*                                                          *
*                 *                       *                *
*                   *        * *        *                  *
*                     * * * * 5 * * * *                    *
*                   *        * *        *                  *
*                 *                       *                *
*                                                          *
*         VERSION 2.3.3                 2015-10-25         *
*                                                          *
*    The MadGraph5_aMC@NLO Development Team - Find us at   *
*    https://server06.fynu.ucl.ac.be/projects/madgraph     *
*                       

INFO: Crossed process found for u c~ > h2 h2 b d~, reuse diagrams. 
INFO: Crossed process found for u c~ > h2 h2 b s~, reuse diagrams. 
INFO: Crossed process found for u c~ > h2 h2 b b~, reuse diagrams. 
INFO: Crossed process found for u d~ > h2 h2 u d~, reuse diagrams. 
INFO: Crossed process found for u d~ > h2 h2 u s~, reuse diagrams. 
INFO: Crossed process found for u d~ > h2 h2 u b~, reuse diagrams. 
INFO: Crossed process found for u d~ > h2 h2 c d~, reuse diagrams. 
INFO: Crossed process found for u d~ > h2 h2 c s~, reuse diagrams. 
INFO: Crossed process found for u d~ > h2 h2 c b~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 u d~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 u s~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 u b~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 c d~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 c s~, reuse diagrams. 
INFO: Crossed process found for u s~ > h2 h2 c b

INFO: Crossed process found for d d~ > h2 h2 g g, reuse diagrams. 
INFO: Crossed process found for d d~ > h2 h2 u u~, reuse diagrams. 
INFO: Crossed process found for d d~ > h2 h2 u c~, reuse diagrams. 
INFO: Crossed process found for d d~ > h2 h2 c u~, reuse diagrams. 
INFO: Crossed process found for d d~ > h2 h2 c c~, reuse diagrams. 
INFO: Crossed process found for d d~ > h2 h2 d d~, reuse diagrams. 
INFO: Crossed process found for d d~ > h2 h2 s s~, reuse diagrams. 
INFO: Crossed process found for d d~ > h2 h2 b b~, reuse diagrams. 
INFO: Crossed process found for d s~ > h2 h2 u u~, reuse diagrams. 
INFO: Crossed process found for d s~ > h2 h2 u c~, reuse diagrams. 
INFO: Crossed process found for d s~ > h2 h2 c u~, reuse diagrams. 
INFO: Crossed process found for d s~ > h2 h2 c c~, reuse diagrams. 
INFO: Crossed process found for d s~ > h2 h2 d s~, reuse diagrams. 
INFO: Crossed process found for d b~ > h2 h2 u u~, reuse diagrams. 
INFO: Crossed process found for d b~ > h2 h2 u c~

INFO: Crossed process found for b b~ > h2 h2 b b~, reuse diagrams. 
INFO: Process u~ g > h2 h2 g u~ added to mirror process g u~ > h2 h2 g u~ 
INFO: Process u~ u > h2 h2 g g added to mirror process u u~ > h2 h2 g g 
INFO: Process u~ u > h2 h2 u u~ added to mirror process u u~ > h2 h2 u u~ 
INFO: Process u~ u > h2 h2 c c~ added to mirror process u u~ > h2 h2 c c~ 
INFO: Process u~ u > h2 h2 d d~ added to mirror process u u~ > h2 h2 d d~ 
INFO: Process u~ u > h2 h2 d s~ added to mirror process u u~ > h2 h2 d s~ 
INFO: Process u~ u > h2 h2 d b~ added to mirror process u u~ > h2 h2 d b~ 
INFO: Process u~ u > h2 h2 s d~ added to mirror process u u~ > h2 h2 s d~ 
INFO: Process u~ u > h2 h2 s s~ added to mirror process u u~ > h2 h2 s s~ 
INFO: Process u~ u > h2 h2 s b~ added to mirror process u u~ > h2 h2 s b~ 
INFO: Process u~ u > h2 h2 b d~ added to mirror process u u~ > h2 h2 b d~ 
INFO: Process u~ u > h2 h2 b s~ added to mirror process u u~ > h2 h2 b s~ 
INFO: Process u~ u > h2 h2 b b~ ad

INFO: Crossed process found for c~ b~ > h2 h2 c~ b~, reuse diagrams. 
INFO: Process d~ g > h2 h2 g d~ added to mirror process g d~ > h2 h2 g d~ 
INFO: Process d~ u > h2 h2 u d~ added to mirror process u d~ > h2 h2 u d~ 
INFO: Process d~ u > h2 h2 u s~ added to mirror process u d~ > h2 h2 u s~ 
INFO: Process d~ u > h2 h2 u b~ added to mirror process u d~ > h2 h2 u b~ 
INFO: Process d~ u > h2 h2 c d~ added to mirror process u d~ > h2 h2 c d~ 
INFO: Process d~ u > h2 h2 c s~ added to mirror process u d~ > h2 h2 c s~ 
INFO: Process d~ u > h2 h2 c b~ added to mirror process u d~ > h2 h2 c b~ 
INFO: Process d~ c > h2 h2 u d~ added to mirror process c d~ > h2 h2 u d~ 
INFO: Process d~ c > h2 h2 u s~ added to mirror process c d~ > h2 h2 u s~ 
INFO: Process d~ c > h2 h2 u b~ added to mirror process c d~ > h2 h2 u b~ 
INFO: Process d~ c > h2 h2 c d~ added to mirror process c d~ > h2 h2 c d~ 
INFO: Process d~ c > h2 h2 c s~ added to mirror process c d~ > h2 h2 c s~ 
INFO: Process d~ c > h2 h2 c b

...

INFO: Generating Feynman diagrams for Process: u~ b~ > h2 h2 c~ d~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group ub_h2h2cd 
INFO: Creating files in directory P0_ub_h2h2cs 
INFO: Generating Feynman diagrams for Process: u b > h2 h2 c s WEIGHTED=4 
INFO: Generating Feynman diagrams for Process: u b~ > h2 h2 c s~ WEIGHTED=4 
INFO: Generating Feynman diagrams for Process: u~ b~ > h2 h2 c~ s~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group ub_h2h2cs 
INFO: Creating files in directory P0_ub_h2h2cb 
INFO: Generating Feynman diagrams for Process: u b > h2 h2 c b WEIGHTED=4 
INFO: Generating Feynman diagrams for Process: u b~ > h2 h2 c b~ WEIGHTED=4 
INFO: Generating Feynman diagrams for Process: u~ b~ > h2 h2 c~ b~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group ub_h2h2cb 
INFO: Creating files in directory P0_uux_h2h2dsx 
INFO: Generating Feynman diagrams for Process: u u~ > h2 h2 d s~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subp

INFO: Finding symmetric diagrams for subprocess group cb_h2h2cd 
INFO: Creating files in directory P0_cb_h2h2cs 
INFO: Generating Feynman diagrams for Process: c b > h2 h2 c s WEIGHTED=4 
INFO: Generating Feynman diagrams for Process: c b~ > h2 h2 c s~ WEIGHTED=4 
INFO: Generating Feynman diagrams for Process: c~ b~ > h2 h2 c~ s~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group cb_h2h2cs 
INFO: Creating files in directory P0_cux_h2h2ddx 
INFO: Generating Feynman diagrams for Process: c u~ > h2 h2 d d~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group cux_h2h2ddx 
INFO: Creating files in directory P0_cux_h2h2dsx 
INFO: Generating Feynman diagrams for Process: c u~ > h2 h2 d s~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group cux_h2h2dsx 
INFO: Creating files in directory P0_cux_h2h2dbx 
INFO: Generating Feynman diagrams for Process: c u~ > h2 h2 d b~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group cux_h2h2dbx 
INFO: Cre

INFO: Generating Feynman diagrams for Process: s c~ > h2 h2 d u~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group scx_h2h2dux 
INFO: Creating files in directory P0_scx_h2h2dcx 
INFO: Generating Feynman diagrams for Process: s c~ > h2 h2 d c~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group scx_h2h2dcx 
INFO: Creating files in directory P0_scx_h2h2sux 
INFO: Generating Feynman diagrams for Process: s c~ > h2 h2 s u~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group scx_h2h2sux 
INFO: Creating files in directory P0_scx_h2h2bux 
INFO: Generating Feynman diagrams for Process: s c~ > h2 h2 b u~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group scx_h2h2bux 
INFO: Creating files in directory P0_scx_h2h2bcx 
INFO: Generating Feynman diagrams for Process: s c~ > h2 h2 b c~ WEIGHTED=4 
INFO: Finding symmetric diagrams for subprocess group scx_h2h2bcx 
INFO: Creating files in directory P0_sdx_h2h2uux 
INFO: Generating Feynman diagr

INFO: copy ../MadGraph_cards/benchmarks/param_card_template.dat as param_card.dat 
INFO: copy ../Cards/run_card.dat as run_card.dat 
set wa0 auto
INFO: modify param_card information BLOCK decay with id (36,) set to Auto 
set whch auto
INFO: modify param_card information BLOCK decay with id (37,) set to Auto 
set lamL 0.01
INFO: modify param_card information BLOCK frblock with id (10,) set to 0.01 
set mmh0 240
INFO: modify param_card information BLOCK frblock with id (13,) set to 240.0 
set mma0 750
INFO: modify param_card information BLOCK frblock with id (14,) set to 750.0 
set mmhch 750
INFO: modify param_card information BLOCK frblock with id (15,) set to 750.0 
Generating 1000 events with run name run_01
survey  run_01 
INFO: compile directory 
INFO: Computing the width set on auto in the param_card.dat 
INFO: Change particles name to pass to MG5 convention 
************************************************************
*                                                          *
* 

INFO:     P0_sux_h2h2bux  
INFO:     P0_sux_h2h2bcx  
INFO:     P0_scx_h2h2dux  
INFO:     P0_scx_h2h2dcx  
INFO:     P0_scx_h2h2sux  
INFO:     P0_scx_h2h2bux  
INFO:     P0_scx_h2h2bcx  
INFO:     P0_sdx_h2h2uux  
INFO:     P0_sdx_h2h2ucx  
INFO:     P0_sdx_h2h2cux  
INFO:     P0_sdx_h2h2ccx  
INFO:     P0_ssx_h2h2ucx  
INFO:     P0_ssx_h2h2cux  
INFO:     P0_sbx_h2h2uux  
INFO:     P0_sbx_h2h2ucx  
INFO:     P0_sbx_h2h2cux  
INFO:     P0_sbx_h2h2ccx  
INFO:     P0_bux_h2h2dux  
INFO:     P0_bux_h2h2dcx  
INFO:     P0_bux_h2h2sux  
INFO:     P0_bux_h2h2scx  
INFO:     P0_bux_h2h2bcx  
INFO:     P0_bcx_h2h2dux  
INFO:     P0_bcx_h2h2dcx  
INFO:     P0_bcx_h2h2sux  
INFO:     P0_bcx_h2h2scx  
INFO:     P0_bcx_h2h2bux  
INFO:     P0_bdx_h2h2uux  
INFO:     P0_bdx_h2h2ucx  
INFO:     P0_bdx_h2h2cux  
INFO:     P0_bdx_h2h2ccx  
INFO:     P0_bsx_h2h2uux  
INFO:     P0_bsx_h2h2ucx  
INFO:     P0_bsx_h2h2cux  
INFO:     P0_bsx_h2h2ccx  
INFO:     P0_bbx_h2h2ucx  
INFO:     P0_bbx_h2h2cux  
I

%%bash
cd madgraph/
./bin/mg5_aMC ../studies/IDM/Task_Asana/BP_750_A_110.txt
cd ..

### Prepare madevent script

In [16]:
pythia_script='TemplateRunPythiaDelphes_all.dat'
if len(LambdasL)>99:
    sys.exit('ERROR: UPDATE FORMAT FOR > 99 runs')
f=open(work_dir+'/'+pythia_script,'w')
for r in range(1,len(LambdasL)+1):
    f.write('pythia run_%02d\n' %r)
    f.write('3\n')
    f.write('0\n')
f.close()

In [18]:
f=open(output_dir+'/kk.sh','w')
f.write('source '+thisroot+'\n')
f.write('./bin/madevent ../'+run_dir+'/'+pythia_script+'\n')
f.close()

In [19]:
#if not VERBOSE:
s=subprocess.Popen('bash kk.sh'.split(), cwd=output_dir,
                                    stdout=subprocess.PIPE, stderr=subprocess.PIPE)

s.wait()
(PHOUT,PHERR)=s.communicate()

print(PHOUT.decode('utf-8'))

In [20]:
cs_pb=np.array( re.sub( '\s+\+\-\s+[0-9\+\-eE\.]+\s+pb','\n',  
          re.sub('\s+Cross-section\s+:\s+','' ,
          ''.join( grep('Cross-section',PHOUT.decode('utf-8')).split('\n') 
          ) )  ).strip().split('\n')  ).astype(float64)

In [21]:
if len(cs_pb)==len(LambdasL):
    df=pd.DataFrame({'xs_'+str(int(MH0)):cs_pb,'laL':LambdasL})
else:
    sys.exit('Error: missing cross section')

In [22]:
df

,laL,xs_240
0,0.01,0.001752
1,0.02,0.001755
2,0.05,0.001776
3,0.07,0.001804
4,0.10,0.001843
5,0.15,0.001883
6,0.20,0.001944
7,0.25,0.001983
8,0.30,0.002075
9,0.35,0.002111


In [23]:
df.to_csv('output/cs_'+str(int(MHc))+'_'+str(int(MH0))+'.csv',index=False)

### Get results

In [33]:
full_output='output'
s=subprocess.Popen(['mkdir', '-p',full_output]).wait()

#if 'xs_'+str(int(MH0)) not in df.columns.values:
if True:
    for r in range(1,len(LambdasL)+1):
        nrun='%02d' %r
        nrun3='%03d' %r
        s=subprocess.Popen(['cp',output_dir+'/Events/run_'+nrun+'/tag_1_delphes_events.root', 
                            full_output+'/delphes_events_'+str(int(MH0))+'_'+nrun3+'_.root'],
                             stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        if s.wait()>0:
            sys.exit('Files not found')

ls studies/IDM/BP_750_110_vs_lambdaL/Events/run_07/

### Merge masses

In [24]:
df_full=pd.read_csv('Output_data.csv')

In [25]:
dff=df_full.merge(df,on='laL',how='left').fillna(0)

In [26]:
tmp=dff['laL']
dff=dff.drop('laL',axis='columns')
dff['laL']=tmp

In [27]:
dff

,Unnamed: 0,xs_70,xs_75,xs_80,xs_85,xs_90,xs_110,xs_130,xs_150,xs_170,xs_190,xs_210,xs_220,xs_240_x,xs_240_y,laL
0,0,0.007836,0.007475,0.007075,0.006754,0.006451,0.005417,0.004505,0.003759,0.003205,0.002678,0.002277,0.002097,0.001752,0.001752,0.01
1,1,0.008207,0.007771,0.007350,0.006937,0.006623,0.005523,0.004599,0.003848,0.003219,0.002666,0.002247,0.002064,0.001755,0.001755,0.02
2,2,0.009770,0.008817,0.008136,0.007636,0.007198,0.005774,0.004747,0.003979,0.003283,0.002781,0.002315,0.002109,0.001776,0.001776,0.05
3,3,0.011421,0.009917,0.008990,0.008275,0.007629,0.006041,0.004918,0.004074,0.003398,0.002825,0.002356,0.002160,0.001804,0.001804,0.07
4,4,0.014126,0.011538,0.010077,0.009139,0.008302,0.006406,0.005106,0.004190,0.003460,0.002875,0.002400,0.002185,0.001843,0.001843,0.10
5,5,0.019924,0.015071,0.012550,0.010998,0.009767,0.007094,0.005427,0.004425,0.003637,0.002974,0.002492,0.002259,0.001883,0.001883,0.15
6,6,0.027188,0.019742,0.015811,0.013316,0.011713,0.007997,0.006022,0.004730,0.003859,0.003159,0.002543,0.002340,0.001944,0.001944,0.20
7,7,0.036839,0.025211,0.019486,0.016115,0.013757,0.008816,0.006455,0.005012,0.004055,0.003234,0.002655,0.002430,0.001983,0.001983,0.25
8,8,0.047742,0.031698,0.023874,0.019238,0.016198,0.009981,0.007122,0.005388,0.004225,0.003419,0.002787,0.002509,0.002075,0.002075,0.30
9,9,0.059759,0.039166,0.028822,0.022678,0.018853,0.011133,0.007755,0.005748,0.004519,0.003553,0.002896,0.002618,0.002111,0.002111,0.35
